In [1]:
import os
import shutil
import random

import cv2 as cv
import numpy as np

# Tools

In [7]:
# Renames
for i in range(1, 1301):
    os.rename(fr"robot\images\left\{i}.jpg", fr"robot\images\left\{i-1}.jpg")

In [17]:
# Move files
path = r"CLICK\H1"
pathSave = r"CLICK\images"
id = 0

dirs = os.listdir(path)
for dir in dirs:
    numberImages = len(os.listdir(fr"{path}\{dir}\data\left"))
    i=0
    while i < numberImages:
        shutil.move(fr"{path}\{dir}\data\left\{i}.jpg", fr"{pathSave}\left\{id}.jpg")
        shutil.move(fr"{path}\{dir}\data\right\{i}.jpg", fr"{pathSave}\right\{id}.jpg")

        id+=1
        i+=1

# CV Flow

In [ ]:
path = fr"CLICK\images"
pathSave = fr"CLICK\flows"
numberImages = len(os.listdir(fr"{path}\left"))

i = 0; error=0; listErrors = []
frame = cv.imread(fr"{path}\left\{i}.jpg")
prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

frame = cv.imread(fr"{path}\right\{i}.jpg")
prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame)
hsv[...,1] = 255
i+=1

while i < numberImages:
    #Left
    frame = cv.imread(fr"{path}\left\{i}.jpg")
    nextL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    flow = cv.calcOpticalFlowFarneback(prvsL,nextL, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    rgb_flowL = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    rgb_flowL = cv.normalize(rgb_flowL, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    #Right
    frame = cv.imread(fr"{path}\right\{i}.jpg")
    nextR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

    flow = cv.calcOpticalFlowFarneback(prvsR,nextR, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    rgb_flowR = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

    rgb_flowR = cv.normalize(rgb_flowR, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    if(np.any(rgb_flowL != 0) and np.any(rgb_flowR != 0)):
        cv.imwrite(f"{pathSave}/left/{i-1}.jpg", rgb_flowL)
        cv.imwrite(f"{pathSave}/right/{i-1}.jpg", rgb_flowR)
        prvsL = nextL
        prvsR = nextR
        i+=1
    else:
        error+=1
        if(error==20):
            listErrors.append(i)
            error=0
            frame = cv.imread(fr"{path}\left\{i}.jpg")
            prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            frame = cv.imread(fr"{path}\right\{i}.jpg")
            prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            i+=1

In [ ]:
# Error correction
path = fr"CLICK\images"
pathSave = fr"CLICK\flows"
numberTry = 0

while len(listErrors) != 0 or numberTry == 20:
    tmp = []
    for i in listErrors:
        # Left
        frame = cv.imread(fr"{path}\left\{i-1}.jpg")
        prvsL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

        hsv = np.zeros_like(frame)
        hsv[...,1] = 255

        frame = cv.imread(fr"{path}\left\{i}.jpg")
        nextL = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

        flow = cv.calcOpticalFlowFarneback(prvsL,nextL, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        rgb_flowL = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

        rgb_flowL = cv.normalize(rgb_flowL, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

        # Right
        frame = cv.imread(fr"{path}\right\{i-1}.jpg")
        prvsR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

        hsv = np.zeros_like(frame)
        hsv[...,1] = 255

        frame = cv.imread(fr"{path}\right\{i}.jpg")
        nextR = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)

        flow = cv.calcOpticalFlowFarneback(prvsR,nextR, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        rgb_flowR = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)

        rgb_flowR = cv.normalize(rgb_flowR, dst=None, alpha=0, beta=255,norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

        if(np.any(rgb_flowL != 0) and np.any(rgb_flowR != 0)):
            cv.imwrite(f"{pathSave}/left/{i-1}.jpg", rgb_flowL)
            cv.imwrite(f"{pathSave}/right/{i-1}.jpg", rgb_flowR)
            prvsL = nextL
            prvsR = nextR
        else:
            tmp.append(i)

    numberTry+=1
    listErrors = tmp



# Wrong pairs

In [4]:
pathGood = r"ISIA\images\good"
pathWrong = r"ISIA\images\wrong"

id = 0
images = os.listdir(fr"{pathGood}/left")
numberImages = len(images)

for i in range(numberImages):
    frame = 0
    while(frame == 0 or f"{secondImage}.jpg" not in images):
        frame = random.randint(-10,10)
        secondImage = i
        secondImage+=frame

    shutil.copyfile(fr"{pathGood}/left/{i}.jpg", fr"{pathWrong}/left/{i}.jpg")
    shutil.copyfile(fr"{pathGood}/right/{secondImage}.jpg", fr"{pathWrong}/right/{i}.jpg")